In [1]:
import pandas as pd

In [5]:
def combine_year(df_list, year, path):
    num = 1
    while num:
        try:
            temp_df = pd.read_csv(path + f'mlb_war_ratings_{year}{num}.csv')
            df_list.append(temp_df)
            num += 1
        except:
            return df_list
            break

In [7]:
df_list = []
path = 'unclean_data/mlb_war_by_year/'
years = [i for i in range(2011, 2019)]
for year in years:
    combine_year(df_list, year, path)

In [61]:
df = pd.concat(df_list)

In [62]:
temp = df.drop(df.loc[df['G'] > 0])

KeyError: "['Rk' 'Player' 'WAR/pos' 'Year' 'Age' 'Tm' 'Lg' 'G' 'PA' 'AB' 'R' 'H' '2B'\n '3B' 'HR' 'RBI' 'BB' 'IBB' 'SO' 'HBP' 'SH' 'SF' 'GDP' 'SB' 'CS' 'BA'\n 'OBP' 'SLG' 'OPS' 'Pos'] not found in axis"

In [13]:
temp = df

In [60]:
df.dropna(how = 'any', subset = ['WAR/pos'], inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df = df[['Player', 'WAR/pos', 'Year']]
df.head()

,Player,WAR/pos,Year
0,Jose Bautista\bautijo02,8.3,2011
1,Jacoby Ellsbury\ellsbja01,8.3,2011
2,Matt Kemp\kempma01,8.0,2011
3,Dustin Pedroia\pedrodu01,8.0,2011
4,Ryan Braun\braunry02,7.7,2011


In [21]:
for i in range(len(df)):
    df.iat[i, 0] = df.iat[i, 0].split('\\')[0]

In [24]:
df = df.rename(columns = {'WAR/pos': 'war', 'Year':'year', 'Player':'player'})

In [26]:
df.to_csv('mlb_player_war_year.csv', index = False)

In [35]:
df.head()

,player,war,year
0,Jose Bautista,8.3,2011
1,Jacoby Ellsbury,8.3,2011
2,Matt Kemp,8.0,2011
3,Dustin Pedroia,8.0,2011
4,Ryan Braun,7.7,2011


In [33]:
mlb_states = pd.read_csv('../static/data/mlb_war_player_state.csv').drop(columns = 'Unnamed: 0')[['Player', 'Year', 'state']]

In [38]:
mlb_states.rename(columns = {'Player':'player', 'Year':'year'}, inplace = True)

In [41]:
len(df)

9798

In [43]:
merged = mlb_states.merge(df, how = 'left')

In [45]:
merged[['year', 'state', 'war']].to_csv('../static/data/year_state_war.csv', index = False)

In [64]:
merged.groupby(['year', 'state']).mean().reset_index().loc[merged['war'] > 0].to_csv('../static/data/year_state_war_avg.csv', index = False)

In [66]:
x = merged.loc[merged['war'] > 0]

In [70]:
x.groupby(['year', 'state']).mean().reset_index().to_csv('../static/data/year_state_war_avg.csv', index = False)

In [92]:
nfl = pd.read_csv('../static/data/year_state_av_avg.csv')
mlb = pd.read_csv('../static/data/year_state_war_avg.csv')
nba = pd.read_csv('../static/data/year_state_per_avg.csv')

In [94]:
df = nba
nba['war'] = 0
nba['av'] = 0

In [132]:
for i in range(len(nba)):
    year = df.iat[i, 0]
    state = df.iat[i, 1]
    if (mlb.loc[(mlb['state'] == state) & (mlb['year'] == year)].empty):
        print()
    else:
        df.iat[i, 3] = mlb.loc[(mlb['state'] == state) & (mlb['year'] == year)].iat[0, 2]
        
    if (nfl.loc[(nfl['state'] == state) & (nfl['year'] == year)].empty):
        print()
    else:
        df.iat[i, 4] = nfl.loc[(nfl['state'] == state) & (nfl['year'] == year)].iat[0, 2]
    

In [135]:
nba.to_csv('../static/data/year_state_sm_avg.csv', index = False)

In [186]:
year = 2012
state = 'TX'

In [125]:
mlb.loc[(mlb['state'] == state) & (mlb['year'] == year)].empty

True

In [136]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [164]:
df = pd.DataFrame(columns = {'states', 'year'}, index = [i for i in range(len(states))])

In [165]:
len(df)

51

In [167]:
df['states'] = states

In [170]:
df = pd.concat([df, df, df, df, df, df, df, df])

In [171]:
num = 0
for i in range(len(df)):
    if i % 51 == 0:
        num += 1
    df.iat[i, 1] = 2010+num

In [188]:
df['per'] = 0.0
df['war'] = 0.0
df['av'] = 0.0
df.head()

,states,year,per,war,av
0,AL,2011,0.0,0.0,0.0
1,AK,2011,0.0,0.0,0.0
2,AZ,2011,0.0,0.0,0.0
3,AR,2011,0.0,0.0,0.0
4,CA,2011,0.0,0.0,0.0


In [189]:
for i in range(len(df)):
    year = df.iat[i, 1]
    state = df.iat[i, 0]
    
    if not (mlb.loc[(mlb['state'] == state) & (mlb['year'] == year)].empty):
        df.iat[i, 3] = mlb.loc[(mlb['state'] == state) & (mlb['year'] == year)].iat[0, 2]
        
    if not (nfl.loc[(nfl['state'] == state) & (nfl['year'] == year)].empty):
        df.iat[i, 4] = nfl.loc[(nfl['state'] == state) & (nfl['year'] == year)].iat[0, 2]
        
    if not (nba.loc[(nba['state'] == state) & (nba['year'] == year)].empty):
        df.iat[i, 2] = nba.loc[(nba['state'] == state) & (nba['year'] == year)].iat[0, 2]

In [191]:
df.to_csv('../static/data/year_state_sm_avg.csv', index = False)

In [187]:
 mlb.loc[(mlb['state'] == state) & (mlb['year'] == year)].iat[0, 2]

1.7120000000000004

In [190]:
df

,states,year,per,war,av
0,AL,2011,13.266667,1.587500,3.741573
1,AK,2011,0.000000,0.000000,2.750000
2,AZ,2011,12.514286,2.186667,3.027778
3,AR,2011,10.966667,0.933333,3.750000
4,CA,2011,13.584091,1.265455,3.153226
5,CO,2011,12.600000,0.000000,3.161290
6,CT,2011,12.988889,0.700000,2.000000
7,DC,2011,8.950000,0.000000,0.000000
8,DE,2011,0.000000,0.000000,0.333333
9,FL,2011,11.745000,2.072000,3.639456
